In [29]:
import pandas as pd

In [30]:
df_game_base = pd.read_csv('game.csv')
df_game_base = df_game_base.dropna(how='all', axis=0, subset=['wl_home'])
# on garde que la "Regular Season"
df_game = df_game_base[df_game_base['season_type'].isin(['Regular Season'])]
# on garde que les années supérieures à 2010
df_game= df_game[df_game['season_id'] >= 21989]
# on garde que les colonnes suivantes : 'season_id', 'team_name_home', 'wl_home', 'team_name_away', 'wl_away'
df_game = df_game.loc[:,['season_id', 'team_name_home', 'wl_home', 'team_name_away', 'wl_away']]
df_game

,season_id,team_name_home,wl_home,team_name_away,wl_away
23329,21989,Phoenix Suns,W,Golden State Warriors,L
23330,21989,Dallas Mavericks,L,Los Angeles Lakers,W
23331,21989,Seattle SuperSonics,W,Minnesota Timberwolves,L
23332,21989,Detroit Pistons,W,New York Knicks,L
23333,21989,Los Angeles Clippers,W,Houston Rockets,L
...,...,...,...,...,...
65537,22022,New York Knicks,L,Indiana Pacers,W
65538,22022,Brooklyn Nets,L,Philadelphia 76ers,W
65539,22022,Chicago Bulls,W,Detroit Pistons,L
65540,22022,Toronto Raptors,W,Milwaukee Bucks,L


In [31]:
# conversion de season_id en année
def supprimer_premier_2(valeur):
    valeur_str = str(valeur)
    return int(valeur_str.replace('2', '', 1)) if '2' in valeur_str else valeur

df_game['season'] = df_game['season_id'].apply(supprimer_premier_2)
df_game['season'] = pd.to_datetime(df_game['season'], format='%Y').dt.year
df_game

,season_id,team_name_home,wl_home,team_name_away,wl_away,season
23329,21989,Phoenix Suns,W,Golden State Warriors,L,1989
23330,21989,Dallas Mavericks,L,Los Angeles Lakers,W,1989
23331,21989,Seattle SuperSonics,W,Minnesota Timberwolves,L,1989
23332,21989,Detroit Pistons,W,New York Knicks,L,1989
23333,21989,Los Angeles Clippers,W,Houston Rockets,L,1989
...,...,...,...,...,...,...
65537,22022,New York Knicks,L,Indiana Pacers,W,2022
65538,22022,Brooklyn Nets,L,Philadelphia 76ers,W,2022
65539,22022,Chicago Bulls,W,Detroit Pistons,L,2022
65540,22022,Toronto Raptors,W,Milwaukee Bucks,L,2022


In [32]:
# total des matchs 'away' gagnés par saison
df_new_game_away = df_game.groupby(['team_name_away', 'season'])['wl_away'].value_counts().reset_index(name='total_away')
df_new_game_away = df_new_game_away.sort_values(by=['team_name_away', 'season', 'wl_away'])
# on conserve que les matchs gagnés
df_new_game_away = df_new_game_away.loc[df_new_game_away['wl_away'] == 'W'].reset_index()
df_new_game_away

,index,team_name_away,season,wl_away,total_away
0,1,Atlanta Hawks,1989,W,16
1,3,Atlanta Hawks,1990,W,14
2,5,Atlanta Hawks,1991,W,15
3,7,Atlanta Hawks,1992,W,18
4,8,Atlanta Hawks,1993,W,21
...,...,...,...,...,...
958,1917,Washington Wizards,2018,W,10
959,1919,Washington Wizards,2019,W,9
960,1921,Washington Wizards,2020,W,15
961,1923,Washington Wizards,2021,W,14


In [33]:
# total des matchs 'home' gagnés par saison
df_new_game_home = df_game.groupby(['team_name_home', 'season'])['wl_home'].value_counts().reset_index(name='total_home')
df_new_game_home = df_new_game_home.sort_values(by=['team_name_home', 'season', 'wl_home'])
# on conserve que les matchs gagnés
df_new_game_home = df_new_game_home.loc[df_new_game_home['wl_home'] == 'W'].reset_index()
df_new_game_home

,index,team_name_home,season,wl_home,total_home
0,0,Atlanta Hawks,1989,W,25
1,2,Atlanta Hawks,1990,W,29
2,4,Atlanta Hawks,1991,W,23
3,6,Atlanta Hawks,1992,W,25
4,8,Atlanta Hawks,1993,W,36
...,...,...,...,...,...
958,1916,Washington Wizards,2018,W,22
959,1919,Washington Wizards,2019,W,16
960,1920,Washington Wizards,2020,W,19
961,1922,Washington Wizards,2021,W,21


In [34]:
df_new_game_home.insert(loc=len(df_new_game_home.columns), column='total_away', value=df_new_game_away['total_away'])
df_new_game_home

,index,team_name_home,season,wl_home,total_home,total_away
0,0,Atlanta Hawks,1989,W,25,16
1,2,Atlanta Hawks,1990,W,29,14
2,4,Atlanta Hawks,1991,W,23,15
3,6,Atlanta Hawks,1992,W,25,18
4,8,Atlanta Hawks,1993,W,36,21
...,...,...,...,...,...,...
958,1916,Washington Wizards,2018,W,22,10
959,1919,Washington Wizards,2019,W,16,9
960,1920,Washington Wizards,2020,W,19,15
961,1922,Washington Wizards,2021,W,21,14


In [35]:
colomns_list = ['total_home', 'total_away']
df_new_game_home['win_match'] = df_new_game_home[colomns_list].sum(axis=1)
df_new_game_home

,index,team_name_home,season,wl_home,total_home,total_away,win_match
0,0,Atlanta Hawks,1989,W,25,16,41
1,2,Atlanta Hawks,1990,W,29,14,43
2,4,Atlanta Hawks,1991,W,23,15,38
3,6,Atlanta Hawks,1992,W,25,18,43
4,8,Atlanta Hawks,1993,W,36,21,57
...,...,...,...,...,...,...,...
958,1916,Washington Wizards,2018,W,22,10,32
959,1919,Washington Wizards,2019,W,16,9,25
960,1920,Washington Wizards,2020,W,19,15,34
961,1922,Washington Wizards,2021,W,21,14,35


In [36]:
# dataframe du total des matchs gagnés par équipe par année

df_new_game = df_new_game_home.loc[:,['team_name_home', 'season', 'win_match']]
df_new_game['Team'] = df_new_game['team_name_home']
df_new_game = df_new_game.loc[:,['Team', 'season', 'win_match']]
df_new_game

,Team,season,win_match
0,Atlanta Hawks,1989,41
1,Atlanta Hawks,1990,43
2,Atlanta Hawks,1991,38
3,Atlanta Hawks,1992,43
4,Atlanta Hawks,1993,57
...,...,...,...
958,Washington Wizards,2018,32
959,Washington Wizards,2019,25
960,Washington Wizards,2020,34
961,Washington Wizards,2021,35


In [37]:
df_draft_history = pd.read_csv('draft_history.csv')

In [38]:
df_draft_history['Team']  = df_draft_history['team_city'] + ' ' + df_draft_history['team_name']
#ajoute d'un décalage de 4 ans
df_draft_history['season'] = df_draft_history.apply(lambda x: x['season'] + 4, axis=1)
# on garde que les années supérieures à 2010
df_draft_history= df_draft_history[df_draft_history['season'] >= 1989]
df_draft_history

,person_id,player_name,season,round_number,round_pick,overall_pick,draft_type,team_id,team_city,team_name,team_abbreviation,organization,organization_type,player_profile_flag,Team
5452,121,Patrick Ewing,1989,1,1,1,Draft,1610612752,New York,Knicks,NYK,Georgetown,College/University,1,New York Knicks
5453,47,Wayman Tisdale,1989,1,2,2,Draft,1610612754,Indiana,Pacers,IND,Oklahoma,College/University,1,Indiana Pacers
5454,104,Benoit Benjamin,1989,1,3,3,Draft,1610612746,Los Angeles,Clippers,LAC,Creighton,College/University,1,Los Angeles Clippers
5455,1365,Xavier McDaniel,1989,1,4,4,Draft,1610612760,Seattle,SuperSonics,SEA,Wichita State,College/University,1,Seattle SuperSonics
5456,402,Jon Koncak,1989,1,5,5,Draft,1610612737,Atlanta,Hawks,ATL,Southern Methodist,College/University,1,Atlanta Hawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7985,1641771,Jalen Slawson,2027,2,24,54,Draft,1610612758,Sacramento,Kings,SAC,Furman,College/University,1,Sacramento Kings
7986,1631209,Isaiah Wong,2027,2,25,55,Draft,1610612754,Indiana,Pacers,IND,Miami (FL),College/University,1,Indiana Pacers
7987,1641844,Tarik Biberovic,2027,2,26,56,Draft,1610612763,Memphis,Grizzlies,MEM,Fenerbahce S.K. (Turkey),Other Team/Club,1,Memphis Grizzlies
7988,1631218,Trayce Jackson-Davis,2027,2,27,57,Draft,1610612764,Washington,Wizards,WAS,Indiana,College/University,1,Washington Wizards


In [39]:
df_draft_history = df_draft_history.loc[:, ['player_name', 'season', 'Team', 'overall_pick']]
df_draft_history

,player_name,season,Team,overall_pick
5452,Patrick Ewing,1989,New York Knicks,1
5453,Wayman Tisdale,1989,Indiana Pacers,2
5454,Benoit Benjamin,1989,Los Angeles Clippers,3
5455,Xavier McDaniel,1989,Seattle SuperSonics,4
5456,Jon Koncak,1989,Atlanta Hawks,5
...,...,...,...,...
7985,Jalen Slawson,2027,Sacramento Kings,54
7986,Isaiah Wong,2027,Indiana Pacers,55
7987,Tarik Biberovic,2027,Memphis Grizzlies,56
7988,Trayce Jackson-Davis,2027,Washington Wizards,57


In [40]:
df_common_player_info = pd.read_csv('common_player_info.csv')

In [41]:

df_common_player_info = df_common_player_info.loc[:,['position', 'display_first_last']]
df_common_player_info = df_common_player_info.fillna('other')
df_common_player_info

,position,display_first_last
0,Forward,Alaa Abdelnaby
1,Center,Kareem Abdul-Jabbar
2,Forward-Guard,Tariq Abdul-Wahad
3,Forward,Shareef Abdur-Rahim
4,Forward,Tom Abernethy
...,...,...
4166,Forward,Paul Zipser
4167,Center,Ante Zizic
4168,Center,Jim Zoet
4169,Center,Ivica Zubac


In [42]:
df_draft = df_draft_history.merge(df_common_player_info, left_on='player_name', right_on='display_first_last')
df_draft

,player_name,season,Team,overall_pick,position,display_first_last
0,Patrick Ewing,1989,New York Knicks,1,Forward,Patrick Ewing
1,Wayman Tisdale,1989,Indiana Pacers,2,Forward,Wayman Tisdale
2,Benoit Benjamin,1989,Los Angeles Clippers,3,Center,Benoit Benjamin
3,Xavier McDaniel,1989,Seattle SuperSonics,4,Forward,Xavier McDaniel
4,Jon Koncak,1989,Atlanta Hawks,5,Center,Jon Koncak
...,...,...,...,...,...,...
1698,Vince Williams Jr.,2026,Memphis Grizzlies,47,Guard,Vince Williams Jr.
1699,Kendall Brown,2026,Minnesota Timberwolves,48,Guard,Kendall Brown
1700,Isaiah Mobley,2026,Cleveland Cavaliers,49,Forward,Isaiah Mobley
1701,Tyrese Martin,2026,Golden State Warriors,51,Guard,Tyrese Martin


In [43]:
# dataframe des joueurs avec leur position et numéro de draft
df_draft = df_draft.loc[:, ['overall_pick', 'season', 'position', 'Team']]
df_draft

,overall_pick,season,position,Team
0,1,1989,Forward,New York Knicks
1,2,1989,Forward,Indiana Pacers
2,3,1989,Center,Los Angeles Clippers
3,4,1989,Forward,Seattle SuperSonics
4,5,1989,Center,Atlanta Hawks
...,...,...,...,...
1698,47,2026,Guard,Memphis Grizzlies
1699,48,2026,Guard,Minnesota Timberwolves
1700,49,2026,Forward,Cleveland Cavaliers
1701,51,2026,Guard,Golden State Warriors


In [44]:
merged_df = pd.merge(df_new_game, df_draft, on=['Team', 'season'], how='outer')
merged_df

,Team,season,win_match,overall_pick,position
0,Atlanta Hawks,1989,41.0,5.0,Center
1,Atlanta Hawks,1989,41.0,41.0,Forward
2,Atlanta Hawks,1989,41.0,59.0,Guard
3,Atlanta Hawks,1989,41.0,77.0,Center
4,Atlanta Hawks,1989,41.0,146.0,Center
...,...,...,...,...,...
1894,Washington Wizards,2023,NaN,9.0,Forward
1895,Washington Wizards,2024,NaN,37.0,Guard
1896,Washington Wizards,2025,NaN,15.0,Forward
1897,Washington Wizards,2026,NaN,10.0,Guard


In [45]:
# merged_df['win_match'] = merged_df['win_match'].interpolate()
merged_df['overall_pick'] = merged_df['overall_pick'].fillna(61)
merged_df.loc[merged_df['overall_pick'] == 61, 'position'] = 'Nothing'
merged_df

,Team,season,win_match,overall_pick,position
0,Atlanta Hawks,1989,41.0,5.0,Center
1,Atlanta Hawks,1989,41.0,41.0,Forward
2,Atlanta Hawks,1989,41.0,59.0,Guard
3,Atlanta Hawks,1989,41.0,77.0,Center
4,Atlanta Hawks,1989,41.0,146.0,Center
...,...,...,...,...,...
1894,Washington Wizards,2023,NaN,9.0,Forward
1895,Washington Wizards,2024,NaN,37.0,Guard
1896,Washington Wizards,2025,NaN,15.0,Forward
1897,Washington Wizards,2026,NaN,10.0,Guard


In [47]:
merged_df.to_csv('dataset_NBA.csv')